# CFTR Modulators and Nutrition

In [14]:
import pandas as pd
import seaborn as sns
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Vigers/CF/Edith Zemanick/CF Nutrition/'
df = pd.read_csv(wd + 'Data_Cleaned/analysis_dataset.csv')
# Make time from Kalydeco or 

## Plots

In [15]:
# At least three annual nutritional measurements prior to first modulator start date, and at least one measurement of nutritional markers and one weight/height measurement >= three months post-modulator
on_effective = df.loc[(df['Modulator'] == 'Trikafta') | (df['Modulator'] == 'Symdeko'),'Patient ID'].unique()
df = df[df['Patient ID'].isin(on_effective)]
# Before or after
df['Before/After'] = df['Age at Test (Years)'] < df['Age at Start (years)']
df['Before/After'].replace({True:'Before',False:'After'},inplace=True)
# Find those matching above criteria
